In [1]:
import geopandas as gpd
import requests
from io import BytesIO
import os
from config import data_folder
from utils.save_shp import save_shp
from utils.load_shp import load_shp

# 1. Extração setores censitários
Nesse notebook é extraída do IBGE a malha de setores censitários com atributos dos resultados de população e domicílios (2022) do estado de São Paulo.

In [2]:
def download_setores_censitarios():

    file = os.path.join(data_folder, 'setores_censitarios.shp')

    if os.path.exists(file):
        print('Retornando arquivo salvo')
        return gpd.read_file(file)

    print('Carregando do IBGE')

    url = ('https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios_preliminares/malha_com_atributos/setores/shp/UF/SP/SP_Malha_Preliminar_2022.zip')

    with requests.get(url, stream = True) as r:
        content = BytesIO(r.content)
        gdf_setores_censitarios = gpd.read_file(content)

    save_shp(gdf_setores_censitarios, 'setores_censitarios.shp')

    return gdf_setores_censitarios

In [3]:
download_setores_censitarios();

Retornando arquivo salvo


# 2. Tratamento dos setores censitários
Nesse notebook é tratada a malha de setores censitários com atributos dos resultados de população e domicílios (2022) do estado de São Paulo.

[Dicionário de dados - IBGE](https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fftp.ibge.gov.br%2FCensos%2FCenso_Demografico_2022%2FAgregados_por_Setores_Censitarios_preliminares%2Farquivos_complementares%2FDicionario_de_dados_agregados_preliminares.xlsx&wdOrigin=BROWSELINK)

In [4]:
setores_censitarios = load_shp('setores_censitarios.shp')

In [5]:
# Coluna de população do setor censitário.
setores_censitarios = setores_censitarios.rename(columns={'v0001': 'pop'})
setores_censitarios['pop'] = setores_censitarios['pop'].astype(int)
setores_censitarios= setores_censitarios.loc[setores_censitarios['pop'] > 0]

In [6]:
# Filtro para o município de São Paulo.
setores_censitarios_sp = setores_censitarios[setores_censitarios['CD_MUN'] == '3550308'].copy()
setores_censitarios_sp['CD_MUN'].unique()
save_shp(setores_censitarios_sp, 'setores_censitarios_sp.shp')

Shapefile salvo em data\setores_censitarios_sp.shp
